## **Aplicação HRNet-Light**

In [1]:
import sys
sys.path.append('../../')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2 as cv
import numpy as np
from torchvision import transforms as T
import json
from pathlib import Path
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import pandas as pd
from tqdm import tqdm

# Importar sua classe existente
from notebooks.playground_VC.hrnet_light.hrnet_light import SimpleHRNet
from src.vfss_dataset import VFSSImageDataset
from src.data_extraction.video_frame import create_video_frame_df, load_video_frame_metadata_from_csv
from src.split_data import split_data_k_fold
from src.utils import get_corners_from_angle
from src.target.heatmap import generate_heatmap_from_points
from src.utils import plot_image_with_mask

In [3]:
# PARTE 1: CONFIGURAÇÃO ESPECÍFICA PARA C2 E C4
# =====================================================================

class C2C4Config:
    """Configurações para detecção de base de C2 e C4"""
    
    # Apenas 2 pontos de interesse
    KEYPOINT_NAMES = [
        'base_c2',  # Base da vértebra C2
        'base_c4'   # Base da vértebra C4
    ]
    
    NUM_KEYPOINTS = 2
    
    # Conexão entre os pontos (linha entre C2 e C4)
    SKELETON = [[0, 1]]  # Conectar base C2 -> base C4
    
    # Dimensões
    IMAGE_SIZE = (256, 256)  # Mesmo da sua classe
    HEATMAP_SIZE = (64, 64)  # 1/4 da imagem para heatmaps
    
    # Cores para visualização (BGR)
    COLORS = {
        'base_c2': (0, 255, 0),    # Verde
        'base_c4': (255, 0, 0),    # Azul
        'skeleton': (0, 255, 255)   # Amarelo
    }
    
    # Caminhos
    MODEL_SAVE_PATH = 'models/hrnet_c2c4.pth'
    CHECKPOINT_DIR = 'checkpoints'


In [ ]:
# PARTE 2: TREINAMENTO
# =====================================================================

class HRNetTrainer:
    """Trainer para HRNet com 2 keypoints (C2 e C4)"""
    
    def __init__(self, model, model_path, best_model_path, device='cuda', learning_rate=1e-3):
        self.model = model.to(device)
        self.device = device

        self.model_path = model_path
        self.best_model_path = best_model_path
        
        # Loss: MSE entre heatmaps
        self.criterion = nn.MSELoss()
        
        # Optimizer
        self.optimizer = torch.optim.Adam(
            self.model.parameters(),
            lr=learning_rate,
            weight_decay=1e-5
        )
        
        # Scheduler
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='min',
            patience=10,
            factor=0.5
        )
        
        # Histórico
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'learning_rate': []
        }
    
    def train_epoch(self, dataloader):
        """Treina uma época"""
        self.model.train()
        total_loss = 0
        
        pbar = tqdm(dataloader, desc='Treinando')
        for images, target, _ in pbar:
            images = images.to(self.device)
            images_resized = F.interpolate(
                                    images, 
                                    size=(1024, 1024), 
                                    mode='bilinear', 
                                    align_corners=False
                                )
            gt_heatmaps = target['heatmap'].to(self.device)
            
            # Forward
            pred_heatmaps = self.model(images_resized)
            loss = self.criterion(pred_heatmaps, gt_heatmaps)
            
            # Backward
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        avg_loss = total_loss / len(dataloader)
        return avg_loss
    
    def validate(self, dataloader):
        """Valida o modelo"""
        self.model.eval()
        total_loss = 0
        
        with torch.no_grad():
            for images, target, _ in tqdm(dataloader, desc='Validando'):
                images = images.to(self.device)
                images_resized = F.interpolate(
                                    images, 
                                    size=(1024, 1024), 
                                    mode='bilinear', 
                                    align_corners=False
                                )
                gt_heatmaps = target['heatmap'].to(self.device)
                
                pred_heatmaps = self.model(images_resized)
                loss = self.criterion(pred_heatmaps, gt_heatmaps)
                
                total_loss += loss.item()
        
        avg_loss = total_loss / len(dataloader)
        return avg_loss
    
    def train(self, train_loader, val_loader, num_epochs=100, save_best=True):
        """Loop de treinamento completo"""
        best_val_loss = float('inf')
        
        print(f"\n{'='*60}")
        print(f"Iniciando treinamento por {num_epochs} épocas")
        print(f"{'='*60}\n")
        
        for epoch in range(num_epochs):
            print(f"\nÉpoca {epoch+1}/{num_epochs}")
            print("-" * 40)
            
            # Treinar
            train_loss = self.train_epoch(train_loader)
            self.history['train_loss'].append(train_loss)
            
            # Validar
            val_loss = self.validate(val_loader)
            self.history['val_loss'].append(val_loss)
            
            # Learning rate atual
            current_lr = self.optimizer.param_groups[0]['lr']
            self.history['learning_rate'].append(current_lr)
            
            # Scheduler step
            self.scheduler.step(val_loss)
            
            # Imprimir resultados
            print(f"\nResultados:")
            print(f"  Train Loss: {train_loss:.4f}")
            print(f"  Val Loss:   {val_loss:.4f}")
            print(f"  LR:         {current_lr:.6f}")
            
            # Salvar melhor modelo
            if save_best and val_loss < best_val_loss:
                best_val_loss = val_loss
                self.save_checkpoint(self.best_model_path)
                print(f"  ✓ Novo melhor modelo salvo! (Val Loss: {val_loss:.4f})")
            
            # Salvar checkpoint regular
            if (epoch + 1) % 10 == 0:
                self.save_checkpoint(f'{self.model_path}_{epoch+1}.pth')
        
        print(f"\n{'='*60}")
        print(f"Treinamento concluído!")
        print(f"Melhor Val Loss: {best_val_loss:.4f}")
        print(f"{'='*60}\n")
        
        return self.history
    
    def save_checkpoint(self, filename):
        """Salva checkpoint do modelo"""
        checkpoint_dir = Path(C2C4Config.CHECKPOINT_DIR)
        checkpoint_dir.mkdir(exist_ok=True)
        
        checkpoint_path = checkpoint_dir / filename
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'history': self.history
        }, checkpoint_path)
    
    def load_checkpoint(self, filename):
        """Carrega checkpoint do modelo"""
        checkpoint_path = Path(C2C4Config.CHECKPOINT_DIR) / filename
        checkpoint = torch.load(checkpoint_path)
        
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.history = checkpoint['history']
        
        print(f"Checkpoint carregado: {filename}")




In [ ]:
# PARTE 3: INFERÊNCIA E VISUALIZAÇÃO
# =====================================================================

class C2C4Predictor:
    """Preditor para C2 e C4"""
    
    def __init__(self, model_path, device='cuda'):
        self.device = device
        self.model = SimpleHRNet(num_keypoints=2)
        
        # Carregar pesos
        checkpoint = torch.load(model_path, map_location=device)
        if 'model_state_dict' in checkpoint:
            self.model.load_state_dict(checkpoint['model_state_dict'])
        else:
            self.model.load_state_dict(checkpoint)
        
        self.model.to(device)
        self.model.eval()
        
        print(f"✓ Modelo carregado: {model_path}")
    
    def heatmaps_to_keypoints(self, heatmaps):
        """
        Converte heatmaps para coordenadas de keypoints
        
        Args:
            heatmaps: (num_keypoints, H, W)
        
        Returns:
            keypoints: np.array [[x, y, conf], [x, y, conf]]
        """
        keypoints = []
        
        for i in range(heatmaps.shape[0]):
            heatmap = heatmaps[i].cpu().numpy()
            
            # Encontrar máximo
            max_idx = np.argmax(heatmap)
            y, x = np.unravel_index(max_idx, heatmap.shape)
            confidence = heatmap[y, x]
            
            # Escalar para tamanho da imagem (256x256)
            scale_x = C2C4Config.IMAGE_SIZE[1] / C2C4Config.HEATMAP_SIZE[1]
            scale_y = C2C4Config.IMAGE_SIZE[0] / C2C4Config.HEATMAP_SIZE[0]
            
            x_scaled = x * scale_x
            y_scaled = y * scale_y
            
            keypoints.append([x_scaled, y_scaled, confidence])
        
        return np.array(keypoints)
    
    @torch.no_grad()
    def predict(self, image_tensor):
        """
        Prediz C2 e C4 em uma imagem
        
        Args:
            image_tensor: (3, 256, 256) ou (B, 3, 256, 256)
        
        Returns:
            keypoints: np.array [[x_c2, y_c2, conf], [x_c4, y_c4, conf]]
        """
        # Adicionar batch dimension se necessário
        if image_tensor.dim() == 3:
            image_tensor = image_tensor.unsqueeze(0)
        
        image_tensor = image_tensor.to(self.device)
        
        # Inferência
        heatmaps = self.model(image_tensor)
        
        # Converter para keypoints
        keypoints = self.heatmaps_to_keypoints(heatmaps[0])
        
        return keypoints
    
    def visualize(self, image, keypoints, save_path=None, show=True):
        """
        Visualiza keypoints detectados
        
        Args:
            image: np.array (H, W, 3) ou torch.Tensor (3, H, W)
            keypoints: np.array [[x, y, conf], [x, y, conf]]
            save_path: caminho para salvar imagem
            show: mostrar com matplotlib
        """
        # Converter tensor para numpy se necessário
        if isinstance(image, torch.Tensor):
            image = image.cpu().numpy()
            if image.shape[0] == 3:  # (3, H, W)
                image = np.transpose(image, (1, 2, 0))
            # Desnormalizar
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            image = std * image + mean
            image = np.clip(image, 0, 1)
        
        # Copiar para desenhar
        img_vis = (image * 255).astype(np.uint8).copy()
        
        # Desenhar keypoints
        for i, (x, y, conf) in enumerate(keypoints):
            if conf > 0.3:  # Threshold de confiança
                color = C2C4Config.COLORS[C2C4Config.KEYPOINT_NAMES[i]]
                
                # Desenhar círculo
                cv.circle(img_vis, (int(x), int(y)), 5, color, -1)
                
                # Desenhar borda
                cv.circle(img_vis, (int(x), int(y)), 6, (0, 0, 0), 2)
                
                # Label
                label = f"{C2C4Config.KEYPOINT_NAMES[i]} ({conf:.2f})"
                cv.putText(
                    img_vis,
                    label,
                    (int(x) + 10, int(y) - 10),
                    cv.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (255, 255, 255),
                    2
                )
                cv.putText(
                    img_vis,
                    label,
                    (int(x) + 10, int(y) - 10),
                    cv.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    color,
                    1
                )
        
        # Desenhar linha conectando C2 e C4
        if keypoints[0][2] > 0.3 and keypoints[1][2] > 0.3:
            pt1 = (int(keypoints[0][0]), int(keypoints[0][1]))
            pt2 = (int(keypoints[1][0]), int(keypoints[1][1]))
            cv.line(img_vis, pt1, pt2, C2C4Config.COLORS['skeleton'], 2)
        
        # Salvar
        if save_path:
            cv.imwrite(save_path, cv.cvtColor(img_vis, cv.COLOR_RGB2BGR))
            print(f"✓ Imagem salva: {save_path}")
        
        # Mostrar
        if show:
            plt.figure(figsize=(10, 10))
            plt.imshow(img_vis)
            plt.axis('off')
            plt.title('Detecção de Base C2 e C4')
            plt.tight_layout()
            plt.show()
        
        return img_vis

def heatmaps_to_keypoints(heatmaps, img_size=256):
    """Converte heatmaps para coordenadas"""
    keypoints = []
    
    for i in range(heatmaps.shape[0]):
        hm = heatmaps[i].cpu().numpy()
        y, x = np.unravel_index(np.argmax(hm), hm.shape)
        conf = hm[y, x]
        
        # Escalar para tamanho da imagem
        x_scaled = x * img_size / hm.shape[1]
        y_scaled = y * img_size / hm.shape[0]
        
        keypoints.append([x_scaled, y_scaled, conf])
    
    return np.array(keypoints)


def test_model(test_loader, model_path='best_model.pth', device='cuda'):
    """Testa o modelo e retorna predições"""
    
    model = SimpleHRNet().to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    print("\n🔍 TESTANDO MODELO")
    print("="*60)
    
    all_predictions = []
    
    with torch.no_grad():
        for images, _ in tqdm(test_loader, desc='Inferência'):
            images = images.to(device)
            heatmaps = model(images)
            
            for i in range(len(images)):
                kpts = heatmaps_to_keypoints(heatmaps[i])
                all_predictions.append({
                    'c2_x': kpts[0][0],
                    'c2_y': kpts[0][1],
                    'c2_conf': kpts[0][2],
                    'c4_x': kpts[1][0],
                    'c4_y': kpts[1][1],
                    'c4_conf': kpts[1][2]
                })
    
    print(f"✓ {len(all_predictions)} predições realizadas")
    
    return pd.DataFrame(all_predictions)


def visualize_prediction(image, keypoints, save_path=None, show=True):
    """Visualiza keypoints detectados"""
    # Desnormalizar imagem
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    img = image * std + mean
    img = (img.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
    
    # Desenhar keypoints
    colors = [(0, 255, 0), (255, 0, 0)]  # C2: verde, C4: azul
    labels = ['C2', 'C4']
    
    for i, (x, y, conf) in enumerate(keypoints):
        cv.circle(img, (int(x), int(y)), 5, colors[i], -1)
        cv.putText(img, f"{labels[i]} ({conf:.2f})", (int(x)+10, int(y)), 
                   cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    # Linha conectando C2-C4
    cv.line(img, (int(keypoints[0][0]), int(keypoints[0][1])),
            (int(keypoints[1][0]), int(keypoints[1][1])), (0, 255, 255), 2)
    
    if save_path:
        cv.imwrite(save_path, cv.cvtColor(img, cv.COLOR_RGB2BGR))
        print(f"✓ Imagem salva: {save_path}")
    
    if show:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10, 10))
        plt.imshow(img)
        plt.axis('off')
        plt.title('Predição C2 e C4')
        plt.tight_layout()
        plt.show()
    
    return img


def visualize_prediction_vs_gt(image_tensor, pred_keypoints, gt_heatmaps, save_path=None, show=True):
    """
    Visualiza predição vs ground truth lado a lado
    
    Args:
        image_tensor: (3, H, W) - imagem normalizada
        pred_keypoints: np.array [[x, y, conf], [x, y, conf]] - predições
        gt_heatmaps: (2, H_hm, W_hm) - heatmaps ground truth
        save_path: caminho para salvar
        show: mostrar com matplotlib
    """
    import matplotlib.pyplot as plt
    
    # Desnormalizar imagem
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    img = image_tensor * std + mean
    img = img.permute(1, 2, 0).cpu().numpy()
    img = np.clip(img, 0, 1)
    
    # Extrair GT keypoints dos heatmaps
    gt_keypoints = []
    img_h, img_w = img.shape[:2]
    hm_h, hm_w = gt_heatmaps.shape[1], gt_heatmaps.shape[2]
    
    for i in range(gt_heatmaps.shape[0]):
        hm = gt_heatmaps[i].cpu().numpy()
        y, x = np.unravel_index(np.argmax(hm), hm.shape)
        
        # Escalar para tamanho da imagem
        x_scaled = x * img_w / hm_w
        y_scaled = y * img_h / hm_h
        conf = hm[y, x]
        
        gt_keypoints.append([x_scaled, y_scaled, conf])
    
    gt_keypoints = np.array(gt_keypoints)
    
    # Criar figura com 2 subplots
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # ========== SUBPLOT 1: Ground Truth ==========
    img_gt = (img * 255).astype(np.uint8).copy()
    
    colors = [(0, 255, 0), (255, 0, 0)]  # C2: verde, C4: vermelho
    labels = ['C2 (GT)', 'C4 (GT)']
    
    for i, (x, y, conf) in enumerate(gt_keypoints):
        cv.circle(img_gt, (int(x), int(y)), 6, colors[i], -1)
        cv.circle(img_gt, (int(x), int(y)), 7, (255, 255, 255), 2)
        cv.putText(img_gt, labels[i], (int(x)+10, int(y)-10),
                   cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # Linha GT
    if len(gt_keypoints) == 2:
        cv.line(img_gt, (int(gt_keypoints[0][0]), int(gt_keypoints[0][1])),
                (int(gt_keypoints[1][0]), int(gt_keypoints[1][1])), (0, 255, 255), 2)
    
    axes[0].imshow(img_gt)
    axes[0].set_title('Ground Truth', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # ========== SUBPLOT 2: Predição ==========
    img_pred = (img * 255).astype(np.uint8).copy()
    
    labels_pred = ['C2 (Pred)', 'C4 (Pred)']
    
    for i, (x, y, conf) in enumerate(pred_keypoints):
        cv.circle(img_pred, (int(x), int(y)), 6, colors[i], -1)
        cv.circle(img_pred, (int(x), int(y)), 7, (255, 255, 255), 2)
        cv.putText(img_pred, f"{labels_pred[i]} ({conf:.2f})", (int(x)+10, int(y)-10),
                   cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # Linha predição
    if len(pred_keypoints) == 2:
        cv.line(img_pred, (int(pred_keypoints[0][0]), int(pred_keypoints[0][1])),
                (int(pred_keypoints[1][0]), int(pred_keypoints[1][1])), (0, 255, 255), 2)
    
    axes[1].imshow(img_pred)
    axes[1].set_title('Predição', fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    # ========== Calcular Erro ==========
    errors = []
    for pred, gt in zip(pred_keypoints, gt_keypoints):
        dist = np.linalg.norm(pred[:2] - gt[:2])
        errors.append(dist)
    
    # Adicionar informação de erro
    error_text = f"Erro C2: {errors[0]:.2f}px | Erro C4: {errors[1]:.2f}px | Erro Médio: {np.mean(errors):.2f}px"
    fig.suptitle(error_text, fontsize=12, y=0.98)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"✓ Visualização salva: {save_path}")
    
    if show:
        plt.show()
    else:
        plt.close()
    
    return fig


def visualize_batch_predictions(images, pred_heatmaps, gt_heatmaps, num_samples=4, save_path=None):
    """
    Visualiza múltiplas predições em grid
    
    Args:
        images: (B, 3, H, W) - batch de imagens
        pred_heatmaps: (B, 2, H_hm, W_hm) - heatmaps preditos
        gt_heatmaps: (B, 2, H_hm, W_hm) - heatmaps ground truth
        num_samples: número de amostras a visualizar
        save_path: caminho para salvar
    """
    import matplotlib.pyplot as plt
    
    num_samples = min(num_samples, len(images))
    
    fig, axes = plt.subplots(num_samples, 2, figsize=(12, 6*num_samples))
    
    if num_samples == 1:
        axes = axes.reshape(1, -1)
    
    for idx in range(num_samples):
        # Extrair dados
        image = images[idx]
        pred_hm = pred_heatmaps[idx]
        gt_hm = gt_heatmaps[idx]
        
        # Converter para keypoints
        pred_kpts = heatmaps_to_keypoints(pred_hm)
        
        # Desnormalizar imagem
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        img = image * std + mean
        img = img.permute(1, 2, 0).cpu().numpy()
        img = np.clip(img, 0, 1)
        
        # GT keypoints
        gt_kpts = []
        img_h, img_w = img.shape[:2]
        hm_h, hm_w = gt_hm.shape[1], gt_hm.shape[2]
        
        for i in range(gt_hm.shape[0]):
            hm = gt_hm[i].cpu().numpy()
            y, x = np.unravel_index(np.argmax(hm), hm.shape)
            x_scaled = x * img_w / hm_w
            y_scaled = y * img_h / hm_h
            gt_kpts.append([x_scaled, y_scaled, hm[y, x]])
        
        gt_kpts = np.array(gt_kpts)
        
        # Desenhar GT
        img_gt = (img * 255).astype(np.uint8).copy()
        for i, (x, y, _) in enumerate(gt_kpts):
            color = (0, 255, 0) if i == 0 else (255, 0, 0)
            cv.circle(img_gt, (int(x), int(y)), 5, color, -1)
        
        axes[idx, 0].imshow(img_gt)
        axes[idx, 0].set_title(f'Sample {idx+1} - Ground Truth', fontweight='bold')
        axes[idx, 0].axis('off')
        
        # Desenhar Predição
        img_pred = (img * 255).astype(np.uint8).copy()
        for i, (x, y, conf) in enumerate(pred_kpts):
            color = (0, 255, 0) if i == 0 else (255, 0, 0)
            cv.circle(img_pred, (int(x), int(y)), 5, color, -1)
        
        # Calcular erro
        error = np.mean([np.linalg.norm(p[:2] - g[:2]) for p, g in zip(pred_kpts, gt_kpts)])
        
        axes[idx, 1].imshow(img_pred)
        axes[idx, 1].set_title(f'Sample {idx+1} - Predição (Erro: {error:.2f}px)', fontweight='bold')
        axes[idx, 1].axis('off')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"✓ Grid salvo: {save_path}")
    
    plt.show()
    
    return fig

In [ ]:
# PARTE 4: AVALIAÇÃO (PCK - Percentage of Correct Keypoints)
# =====================================================================

def calculate_pck(pred_keypoints, gt_keypoints, threshold=0.1, image_size=256):
    """
    Calcula PCK (Percentage of Correct Keypoints)
    
    Args:
        pred_keypoints: np.array (N, 2, 3) - predições
        gt_keypoints: np.array (N, 2, 3) - ground truth
        threshold: percentual da diagonal da imagem
        image_size: tamanho da imagem
    
    Returns:
        pck: float - percentual de keypoints corretos
    """
    diagonal = np.sqrt(2 * image_size**2)
    threshold_pixels = threshold * diagonal
    
    correct = 0
    total = 0
    
    for pred, gt in zip(pred_keypoints, gt_keypoints):
        for i in range(len(pred)):
            if gt[i][2] > 0:  # Se keypoint é visível
                dist = np.linalg.norm(pred[i][:2] - gt[i][:2])
                if dist < threshold_pixels:
                    correct += 1
                total += 1
    
    pck = correct / total if total > 0 else 0
    return pck


def evaluate_model(model, dataloader, device='cuda'):
    """
    Avalia modelo em dataset de validação
    
    Args:
        model: modelo treinado
        dataloader: dataloader de validação
        device: 'cuda' ou 'cpu'
    
    Returns:
        metrics: dict com métricas de avaliação
    """
    model.eval()
    predictor = C2C4Predictor.__new__(C2C4Predictor)
    predictor.model = model
    predictor.device = device
    
    all_pred_keypoints = []
    all_gt_keypoints = []
    
    print("\nAvaliando modelo...")
    with torch.no_grad():
        for images, target_heatmaps, meta in tqdm(dataloader):
            images = images.to(device)
            
            # Predição
            pred_heatmaps = model(images)
            
            # Converter heatmaps para keypoints
            for i in range(len(images)):
                pred_kpts = predictor.heatmaps_to_keypoints(pred_heatmaps[i])
                gt_kpts = meta['keypoints'][i].cpu().numpy()
                
                all_pred_keypoints.append(pred_kpts)
                all_gt_keypoints.append(gt_kpts)
    
    # Calcular métricas
    all_pred_keypoints = np.array(all_pred_keypoints)
    all_gt_keypoints = np.array(all_gt_keypoints)
    
    pck_05 = calculate_pck(all_pred_keypoints, all_gt_keypoints, threshold=0.05)
    pck_10 = calculate_pck(all_pred_keypoints, all_gt_keypoints, threshold=0.10)
    pck_20 = calculate_pck(all_pred_keypoints, all_gt_keypoints, threshold=0.20)
    
    # Calcular NME (Normalized Mean Error)
    distances = []
    for pred, gt in zip(all_pred_keypoints, all_gt_keypoints):
        for i in range(len(pred)):
            if gt[i][2] > 0:
                dist = np.linalg.norm(pred[i][:2] - gt[i][:2])
                distances.append(dist)
    
    nme = np.mean(distances) / 256.0  # Normalizado pelo tamanho da imagem
    
    metrics = {
        'PCK@0.05': pck_05,
        'PCK@0.10': pck_10,
        'PCK@0.20': pck_20,
        'NME': nme,
        'Mean Distance (pixels)': np.mean(distances)
    }
    
    print("\n" + "="*60)
    print("MÉTRICAS DE AVALIAÇÃO")
    print("="*60)
    for metric, value in metrics.items():
        print(f"{metric:25s}: {value:.4f}")
    print("="*60 + "\n")
    
    return metrics

In [ ]:
# PARTE 5: VALIDAÇÃO CRUZADA
# =====================================================================

def cross_validate_hrnet_light(model_class, target, output_dim, folds, device, \
               model_save_path, best_model_save_path, epochs, batch_size, model_kwargs):
    
    k = len(folds)
    for i in range(1):
        # Divide folds: 1 p/ validação, K-1 p/ treino
        df_val = folds[i]
        df_train = [folds[j] for j in range(k) if j != i]
        df_train = pd.concat(df_train, ignore_index=True)

        # Transformando Folds
        train_set = VFSSImageDataset(
            video_frame_df=df_train,
            target=target,
            output_dim=output_dim
            )
        
        val_set = VFSSImageDataset(
            video_frame_df=df_val,
            target=target,
            output_dim=output_dim
            )


        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=batch_size)
        
        # Treinar
        model_save_path_final = model_save_path + f"_fold{i+1}"
        best_model_save_path_final = best_model_save_path + f"_fold{i+1}.pth.tar"
        model = model_class(**model_kwargs)
        trainer = HRNetTrainer(model, model_save_path_final, best_model_save_path_final, device=device)
        trainer.train(train_loader, val_loader, num_epochs=epochs)

In [ ]:
# APLICAÇÃO - INICIALIZAÇÃO 
# =====================================================================

videos_dir = 'C:\\Users\\vinic\\OneDrive\\Documentos\\Faculdade\\Doutorado\\Pesquisa_INCA\\Disfagia_Vinicius\\data\\videos\\' 
labels_dir = 'C:\\Users\\vinic\\OneDrive\\Documentos\\Faculdade\\Doutorado\\Pesquisa_INCA\\vfss-data-split\\data\\rotulos\\anotacoes-tecgraf\\'

# Setta o ambiente e carrega o conjunto de dados
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

video_frame_df = create_video_frame_df(
    videos_dir=videos_dir,
    labels_dir=labels_dir,
    target='points'
)

print("\nSplit Dados")
list_df_folds, df_test = split_data_k_fold(video_frame_df, test_size=0.2, n_folds=5)

# Conjunto de Teste
test_data = VFSSImageDataset(
    video_frame_df=df_test,
    target='heatmap+roi+points',
    output_dim=(256, 256)
)

Shape of attributions metadata df: (2000, 5)
Shape of label metadata df: (1361, 7)
Shape of labeled frames df: (2000, 9)

Split Dados
Folds all: 92
Test: 23
Overall Dataset - Videos: 234 | Patients: 115 | Frames: 814 (100.0%)
Training + Validation Set - Videos: 192 | Patients: 92 | Frames: 668 (82.1%)
Test Set - Videos: 42 | Patients: 23 | Frames: 146 (17.9%)

Fold 1 - Videos: 41 | Patients: 19 | Frames: 145 (17.8%)
Fold 2 - Videos: 56 | Patients: 19 | Frames: 183 (22.5%)
Fold 3 - Videos: 40 | Patients: 18 | Frames: 155 (19.0%)
Fold 4 - Videos: 26 | Patients: 18 | Frames: 95 (11.7%)
Fold 5 - Videos: 29 | Patients: 18 | Frames: 90 (11.1%)


,split_name,n_videos,n_patients,n_frames,percentage_of_initial_frames,video_ids
0,fold_1,41,19,145,0.178133,"[101, 102, 103, 104, 105, 106, 107, 108, 109, ..."
1,fold_2,56,19,183,0.224816,"[129, 130, 131, 132, 136, 137, 138, 139, 140, ..."
2,fold_3,40,18,155,0.190418,"[122, 123, 125, 126, 127, 128, 152, 153, 154, ..."
3,fold_4,26,18,95,0.116708,"[10, 113, 114, 115, 116, 117, 118, 119, 120, 1..."
4,fold_5,29,18,90,0.110565,"[192, 193, 194, 195, 196, 197, 198, 199, 200, ..."
5,test_set,42,23,146,0.179361,"[100, 11, 13, 142, 143, 144, 145, 146, 147, 14..."


In [ ]:
# APLICAÇÃO - TREINAMENTO DO MODELO 
# =====================================================================

model_save_path = "C:\\Users\\vinic\\OneDrive\\Documentos\\Faculdade\\Doutorado\\Pesquisa_INCA\\vfss-data-split\\data\\model_weights\\hrnet\\hrnet_checkpoint"
best_model_save_path = "C:\\Users\\vinic\\OneDrive\\Documentos\\Faculdade\\Doutorado\\Pesquisa_INCA\\vfss-data-split\\data\\model_weights\\hrnet\\hrnet_best"

batch_size = 8         # 8     -> Valor Anterior
epochs = 200            # 100   -> Valor Anterior

target = "heatmap+roi+points"      # Tem que generalizar mais a função
output_dim = (256, 256)

# Divide em treino e validação
generator = torch.Generator().manual_seed(42)


model_kwargs = {
    "num_keypoints": 2,
}

cross_validate_hrnet_light(SimpleHRNet,
               target,
               output_dim,
               list_df_folds,
               device,
               model_save_path,
               best_model_save_path,               
               epochs,
               batch_size,
               model_kwargs
)


Iniciando treinamento por 200 épocas


Época 1/200
----------------------------------------


Validando: 100%|██████████| 19/19 [00:29<00:00,  1.57s/it]



Resultados:
  Train Loss: 12.5841
  Val Loss:   0.8345
  LR:         0.001000
  ✓ Novo melhor modelo salvo! (Val Loss: 0.8345)

Época 2/200
----------------------------------------


Validando: 100%|██████████| 19/19 [00:29<00:00,  1.56s/it]



Resultados:
  Train Loss: 0.4156
  Val Loss:   0.3146
  LR:         0.001000
  ✓ Novo melhor modelo salvo! (Val Loss: 0.3146)

Época 3/200
----------------------------------------


...


Resultados:
  Train Loss: 0.0024
  Val Loss:   0.0019
  LR:         0.000008

Época 199/200
----------------------------------------


Validando: 100%|██████████| 19/19 [00:33<00:00,  1.75s/it]



Resultados:
  Train Loss: 0.0024
  Val Loss:   0.0019
  LR:         0.000008

Época 200/200
----------------------------------------


Validando: 100%|██████████| 19/19 [00:33<00:00,  1.76s/it]


Resultados:
  Train Loss: 0.0024
  Val Loss:   0.0019
  LR:         0.000008

Treinamento concluído!
Melhor Val Loss: 0.0019



In [ ]:
# APLICAÇÃO - SALVANDO VISUALIZAÇÕES
# =====================================================================

for fold_idx in range(1):
    path = best_model_save_path + f"_fold{fold_idx+1}.pth.tar"
    predictor = C2C4Predictor(path)
    
    for idx, (image, target, _) in enumerate(test_data):

        pred_keypoints = predictor.predict(image)
        
        visualize_prediction_vs_gt(
            image, 
            pred_keypoints, 
            target['heatmap'],
            save_path=f'C:\\Users\\vinic\\OneDrive\\Documentos\\Faculdade\\Doutorado\\Pesquisa_INCA\\vfss-data-split\\data\\vis\\hrnet\\fold{fold_idx+1}_sample{idx}.png',
            show=False
        )



✓ Modelo carregado: C:\Users\vinic\OneDrive\Documentos\Faculdade\Doutorado\Pesquisa_INCA\vfss-data-split\data\model_weights\hrnet\hrnet_best_fold1.pth.tar
✓ Visualização salva: C:\Users\vinic\OneDrive\Documentos\Faculdade\Doutorado\Pesquisa_INCA\vfss-data-split\data\vis\hrnet\fold1_sample0.png
✓ Visualização salva: C:\Users\vinic\OneDrive\Documentos\Faculdade\Doutorado\Pesquisa_INCA\vfss-data-split\data\vis\hrnet\fold1_sample1.png
✓ Visualização salva: C:\Users\vinic\OneDrive\Documentos\Faculdade\Doutorado\Pesquisa_INCA\vfss-data-split\data\vis\hrnet\fold1_sample2.png
✓ Visualização salva: C:\Users\vinic\OneDrive\Documentos\Faculdade\Doutorado\Pesquisa_INCA\vfss-data-split\data\vis\hrnet\fold1_sample3.png
✓ Visualização salva: C:\Users\vinic\OneDrive\Documentos\Faculdade\Doutorado\Pesquisa_INCA\vfss-data-split\data\vis\hrnet\fold1_sample4.png
✓ Visualização salva: C:\Users\vinic\OneDrive\Documentos\Faculdade\Doutorado\Pesquisa_INCA\vfss-data-split\data\vis\hrnet\fold1_sample5.png
✓ Vis

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\vinic\\OneDrive\\Documentos\\Faculdade\\Doutorado\\Pesquisa_INCA\\vfss-data-split\\data\\model_weights\\hrnet\\hrnet_best_fold2.pth.tar'